In [2]:
from flask import Flask
import eventlet.wsgi
import eventlet
import socketio
import numpy as np
import base64
from PIL import Image
from io import BytesIO
from tensorflow.keras.models import load_model
import pyvjoy
import pygame

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
# Define some colors.
BLACK = pygame.Color('black')
WHITE = pygame.Color('white')


# This is a simple class that will help us print to the screen.
# It has nothing to do with the joysticks, just outputting the
# information.
class TextPrint(object):
    def __init__(self):
        self.reset()
        self.font = pygame.font.Font(None, 20)

    def tprint(self, screen, textString):
        textBitmap = self.font.render(textString, True, BLACK)
        screen.blit(textBitmap, (self.x, self.y))
        self.y += self.line_height

    def reset(self):
        self.x = 10
        self.y = 10
        self.line_height = 15

    def indent(self):
        self.x += 10

    def unindent(self):
        self.x -= 10


pygame.init()

# Set the width and height of the screen (width, height).
screen = pygame.display.set_mode((500, 700))

pygame.display.set_caption("My Game")

# Loop until the user clicks the close button.
done = False

# Used to manage how fast the screen updates.
clock = pygame.time.Clock()

# Initialize the joysticks.
pygame.joystick.init()

# Get ready to print.
textPrint = TextPrint()
# -------- Main Program Loop -----------
while not done:
    #
    # EVENT PROCESSING STEP
    #
    # Possible joystick actions: JOYAXISMOTION, JOYBALLMOTION, JOYBUTTONDOWN,
    # JOYBUTTONUP, JOYHATMOTION
    for event in pygame.event.get(): # User did something.
        if event.type == pygame.QUIT: # If user clicked close.
            done = True # Flag that we are done so we exit this loop.
        elif event.type == pygame.JOYBUTTONDOWN:
            print("Joystick button pressed.")
        elif event.type == pygame.JOYBUTTONUP:
            print("Joystick button released.")

    #
    # DRAWING STEP
    #
    # First, clear the screen to white. Don't put other drawing commands
    # above this, or they will be erased with this command.
    screen.fill(WHITE)
    textPrint.reset()

    # Get count of joysticks.
    joystick_count = pygame.joystick.get_count()

    textPrint.tprint(screen, "Number of joysticks: {}".format(joystick_count))
    textPrint.indent()

    # For each joystick:
    for i in range(joystick_count):
        joystick = pygame.joystick.Joystick(i)
        joystick.init()

        textPrint.tprint(screen, "Joystick {}".format(i))
        textPrint.indent()

        # Get the name from the OS for the controller/joystick.
        name = joystick.get_name()
        textPrint.tprint(screen, "Joystick name: {}".format(name))

        # Usually axis run in pairs, up/down for one, and left/right for
        # the other.
        axes = joystick.get_numaxes()
        textPrint.tprint(screen, "Number of axes: {}".format(axes))
        textPrint.indent()

        for i in range(axes):
            axis = joystick.get_axis(i)
            textPrint.tprint(screen, "Axis {} value: {:>6.3f}".format(i, axis))
        textPrint.unindent()

        buttons = joystick.get_numbuttons()
        textPrint.tprint(screen, "Number of buttons: {}".format(buttons))
        textPrint.indent()

        for i in range(buttons):
            button = joystick.get_button(i)
            textPrint.tprint(screen,
                             "Button {:>2} value: {}".format(i, button))
        textPrint.unindent()

        hats = joystick.get_numhats()
        textPrint.tprint(screen, "Number of hats: {}".format(hats))
        textPrint.indent()

        # Hat position. All or nothing for direction, not a float like
        # get_axis(). Position is a tuple of int values (x, y).
        for i in range(hats):
            hat = joystick.get_hat(i)
            textPrint.tprint(screen, "Hat {} value: {}".format(i, str(hat)))
        textPrint.unindent()

        textPrint.unindent()

    #
    # ALL CODE TO DRAW SHOULD GO ABOVE THIS COMMENT
    #

    # Go ahead and update the screen with what we've drawn.
    pygame.display.flip()

    # Limit to 20 frames per second.
    clock.tick(20)

# Close the window and quit.
# If you forget this line, the program will 'hang'
# on exit if running from IDLE.
pygame.quit()

Joystick button pressed.
Joystick button released.
Joystick button pressed.
Joystick button released.
Joystick button pressed.
Joystick button released.
Joystick button pressed.
Joystick button released.


In [2]:
sio = socketio.Server()
app = Flask(__name__)

def send_control(steering_angle, throttle):
    sio.emit("steer", data = {"steering_angle": str(steering_angle),
                             "throttle": str(throttle)}, skip_sid = True)

def process_image(image):
    return image[10:130:2, ::4, :]

In [3]:
model = load_model('f:\Pliki ściągnięte\mymodel1 (1).h5')

In [ ]:
@sio.on('telemetry')
def telemetry(sid, data):
    if data:
        speed = float(data["speed"])
        image_str = data["image"]
        
        decoded = base64.b64decode(image_str)
        image = Image.open(BytesIO(decoded))
        image_array = np.asarray(image)
#         print(image_array.shape)
        img = process_image(image_array)
#         print(img.shape)
        img_batch = np.expand_dims(img, axis=0)
#         print(img_batch.shape)

        steering_angle = model.predict(img_batch)
        throttle = 0.3
        if speed < 18:
            throttle = 0.8
        if speed > 21:
            throttle = -0.1

        send_control(steering_angle[0][0], throttle)
    else:
        sio.emit("manual", data = (), skip_sid = True)
        
app = socketio.Middleware(sio, app)
eventlet.wsgi.server(eventlet.listen(('',4567)), app)

(8084) wsgi starting up on http://0.0.0.0:4567
(8084) accepted ('127.0.0.1', 59159)
127.0.0.1 - - [28/Jan/2020 21:05:27] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 116.833108


In [2]:
MAX_VJOY = 32767
NEUTRAL_VJOY = MAX_VJOY/2
j = pyvjoy.VJoyDevice(1)

def play_function(X,Y,Z,XRot):
    j.data.wAxisX = int(X * self.MAX_VJOY)
    j.data.wAxisY = int(Y * self.MAX_VJOY)
    j.data.wAxisZ = int(Z * self.MAX_VJOY)
    j.data.wAxisXRot = int(XRot * self.MAX_VJOY)
    j.update()
    

In [ ]:
j.

In [113]:
MAX_VJOY = 32767
NEUTRAL_VJOY = int(MAX_VJOY/2)
j = pyvjoy.VJoyDevice(1)
j.set_button(1,1)
j.set_button(1,0)
# j.set_axis(pyvjoy.HID_USAGE_X, MAX_VJOY)
# j.set_axis(pyvjoy.HID_USAGE_X, 32767)

j.data.wAxisX = NEUTRAL_VJOY 
j.data.wAxisY= MAX_VJOY

j.data.wAxisXRot = MAX_VJOY 
j.data.wAxisYRot = NEUTRAL_VJOY 
j.update()

1